In [22]:
from bs4 import BeautifulSoup
import requests
import csv 
import os

In [23]:
# URL du site web
url = 'https://www.esb.tn/programmes/masters-professionnels/master-en-management-digital-systemes-dinformation/'
# Charger le contenu HTML depuis la page web
response = requests.get(url)
response.raise_for_status()  # Vérifier que la requête s'est bien passée
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Extraction des sections Objectifs, Contenu et Compétences

# Extraire la section "Objectifs"
objectifs_section = soup.find('h3', text="Objectifs")
if objectifs_section:
    objectifs = objectifs_section.find_next('p').text.strip()  # Premier paragraphe
    objectifs_suite = objectifs_section.find_next('p').find_next('p').text.strip()  # Deuxième paragraphe
    objectifs = f"{objectifs}\n{objectifs_suite}"

    # Rechercher la section "Contenu" en ignorant les espaces insécables
    contenu_section = soup.find('h3', text=lambda t: t and "Contenu" in t)
    if contenu_section:
        # Trouver la balise <ul> immédiatement après
        ul_tag = contenu_section.find_next('ul')
        if ul_tag:
            contenu_list = ul_tag.find_all('li')
            contenu = "\n".join([item.text.strip() for item in contenu_list])
            print("Contenu extrait :")
            
        else:
            print("Aucune balise <ul> trouvée après <h3> Contenu.")
    else:
        print("La section 'Contenu' n'a pas été trouvée.")

# Extraire la section "Compétences"
competences_section = soup.find('h3', text="Compétences")
if competences_section:
    competences_list = competences_section.find_next('ul').find_all('li')
    competences = "\n".join([item.text.strip() for item in competences_list])

# Format final pour l'affichage
result = f"Objectifs\n{objectifs}\n\nContenu\n{contenu}\n\nCompétences\n{competences}"

# Afficher le résultat final
print(result)

Contenu extrait :
Objectifs
> Entreprises de services numériques
> Opérateur de télécommunication

Contenu
MANAGEMENT : Business Process Management, Conduite du Changement, Management de projets, Supply Chain, etc.
DIGITAL : ERP, Business Intelligence, Big Data, SAP, CRM
SOFT SKILLS : Séminaires, Développement Personnel, Business Games, Langues, etc.
PROJET PROFESSIONNEL : Stages, PFE, Projets intégrés, Visites d’entreprises, etc.

Compétences
Anticiper et gérer les changements liés à la transformation digitale.
Identifier les choix stratégiques appropriés pour le management et l’optimisation des systèmes d’information des entreprises .
Intégrer le management des systèmes d’information et des outils technologiques IT dans la stratégie globale de l’entreprise.
Assurer la veille technologique et évaluer la pertinence des technologies émergentes dans le développement des affaires pour une entreprise.
Conduire des projets à fort contenu technologique et s’assurer de la performance économiq

C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\380697594.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  objectifs_section = soup.find('h3', text="Objectifs")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\380697594.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contenu_section = soup.find('h3', text=lambda t: t and "Contenu" in t)
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\380697594.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  competences_section = soup.find('h3', text="Compétences")


In [25]:
# Extraire la section "Métiers"
metiers_section = soup.find('h3', text="Métiers")
if metiers_section:
    metiers_list = metiers_section.find_next('ul').find_all('li')
    metiers = "\n".join([item.text.strip() for item in metiers_list])

# Extraire la section "Secteurs d’activité"
secteurs_section = soup.find('h3', text="Secteurs d’activité")
if secteurs_section:
    secteurs_list = secteurs_section.find_next('ul').find_all('li')
    secteurs = "\n".join([item.text.strip() for item in secteurs_list])

# Extraire la section "Partenariats professionnels"
partenariats_section = soup.find('div', class_='stage')
if partenariats_section:
    partenariats_list = partenariats_section.find_all('p')
    partenariats = "\n".join([item.text.strip().replace('>', '').strip() for item in partenariats_list if item.text.strip()])

# Format final pour l'affichage
result = f"Métiers\n{metiers}\n\nSecteurs d’activité\n{secteurs}\n\nPartenariats professionnels\n{partenariats}"

# Afficher le résultat final
print(result)

Métiers
Responsable de la Transformation Digital : Chief Digital Officer, Responsable Stratégie Digitale
Chef de Projet Digital : Digital Account Executive, IT Project Manager, Chef de Produit / Service, Manager des Actifs Numériques.
Consultant en Systèmes d’Information : Consultant ERP, Consultant IT, Consultant CRM

Secteurs d’activité
Entreprises de services numériques
Sociétés de conseil
Banques & Assurances
BTP
Technologies de l’information et des communications.

Partenariats professionnels
Entreprises de services numériques
Opérateur de télécommunication
Banques et Assurances


C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\1374732328.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  metiers_section = soup.find('h3', text="Métiers")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\1374732328.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  secteurs_section = soup.find('h3', text="Secteurs d’activité")


In [26]:
# Extraction des matières par semestre
semestres_section = soup.find_all('div', class_='elementor-accordion-item')

# Stocker les matières par semestre
matieres_par_semestre = {}

# Variable pour contrôler l'ajout de "Période d'Harmonisation" une seule fois
periode_harmonisation_extracted = False

# Parcourir chaque section de semestre
for semestre in semestres_section:
    # Extraire le nom du semestre (ex: "Période d’Harmonisation", "Semestre 1", etc.)
    semestre_title = semestre.find('a')
    if semestre_title:
        semestre_name = semestre_title.text.strip()

        # Extraire les matières pour ce semestre
        contenu_section = semestre.find('div', class_='elementor-tab-content')
        if contenu_section:
            matieres_list = contenu_section.find_all('li')
            matieres = [item.text.strip() for item in matieres_list]

            # Gérer "Période d'Harmonisation" et éviter sa répétition
            if semestre_name == "Période d’Harmonisation" and not periode_harmonisation_extracted:
                matieres_par_semestre["Période d’Harmonisation"] = matieres
                periode_harmonisation_extracted = True
            elif semestre_name != "Période d’Harmonisation":
                matieres_par_semestre[semestre_name] = matieres

# Format final pour l'affichage
result = "Matières par semestre\n"

# Ajouter la "Période d’Harmonisation" en premier, s'il existe
if "Période d’Harmonisation" in matieres_par_semestre:
    result += f"\nPériode d’Harmonisation\n"
    result += "\n".join(matieres_par_semestre["Période d’Harmonisation"]) + "\n"

# Ajouter les semestres dans l'ordre
for semestre, matieres in matieres_par_semestre.items():
    if semestre != "Période d’Harmonisation":
        result += f"\n{semestre}\n"
        result += "\n".join(matieres) + "\n"

# Afficher le résultat final
print(result)

Matières par semestre

Période d’Harmonisation
Probabilités et analyse statistique
Atelier de programmation et introduction aux SI
Introduction au management et gestion de Projets
Anglais
Français
Séminaires

Période d'Harmonisation
Probabilités et Analyse Statistique
Atelier de programmation et Introduction aux SI
Introduction au Management et Gestion de Projets
Anglais
Français
Séminaires

Semestre 1
Architecture des systèmes d’information
Management des systèmes d’information
Digital technology & business value
Management stratégique et transition digitale
e-Business
Droit du numérique
Anglais
Efficacité professionnelle et développement personnel
Séminaires métiers I
Projet intégré I

Semestre 2
Management des projets et conduite du changement
Veille et intelligence technologique
Transformation digitale & supply chain
Web marketing
Business process management
Analyse des données et prise de décision
Outillage en machine learning
Projet intégré II
Anglais II
Compétences personnelles 

In [27]:
# Extraction des sections

# Objectifs
objectifs_section = soup.find('h3', text="Objectifs")
objectifs = ""
if objectifs_section:
    objectifs = objectifs_section.find_next('p').text.strip()  # Premier paragraphe
    objectifs_suite = objectifs_section.find_next('p').find_next('p').text.strip()  # Deuxième paragraphe
    objectifs = f"{objectifs}\n{objectifs_suite}"

# Compétences
competences_section = soup.find('h3', text="Compétences")
competences = ""
if competences_section:
    competences_list = competences_section.find_next('ul').find_all('li')
    competences = "\n".join([item.text.strip() for item in competences_list])

# Métiers
metiers_section = soup.find('h3', text="Métiers")
metiers = ""
if metiers_section:
    metiers_list = metiers_section.find_next('ul').find_all('li')
    metiers = "\n".join([item.text.strip() for item in metiers_list])

# Secteurs d'activité
secteurs_section = soup.find('h3', text="Secteurs d’activité")
secteurs = ""
if secteurs_section:
    secteurs_list = secteurs_section.find_next('ul').find_all('li')
    secteurs = "\n".join([item.text.strip() for item in secteurs_list])

# Partenariats professionnels
partenariats_section = soup.find('div', class_='stage')
partenariats = ""
if partenariats_section:
    partenariats_list = partenariats_section.find_all('p')
    partenariats = "\n".join([item.text.strip().replace('>', '').strip() for item in partenariats_list if item.text.strip()])

# Matières par semestre
semestres_section = soup.find_all('div', class_='elementor-accordion-item')
matieres_par_semestre = {}
periode_harmonisation_extracted = False

for semestre in semestres_section:
    semestre_title = semestre.find('a')
    if semestre_title:
        semestre_name = semestre_title.text.strip()
        contenu_section = semestre.find('div', class_='elementor-tab-content')
        if contenu_section:
            matieres_list = contenu_section.find_all('li')
            matieres = [item.text.strip() for item in matieres_list]
            if semestre_name == "Période d’Harmonisation" and not periode_harmonisation_extracted:
                matieres_par_semestre["Période d’Harmonisation"] = matieres
                periode_harmonisation_extracted = True
            elif semestre_name != "Période d’Harmonisation":
                matieres_par_semestre[semestre_name] = matieres

# Création des données à insérer dans le CSV
data = {
    'Objectifs': objectifs,
    'Contenu': contenu,
    'Compétences': competences,
    'Métiers': metiers,
    'Secteurs d’activité': secteurs,
    'Partenariats professionnels': partenariats
}

# Ajouter les matières par semestre au dictionnaire des données
for semestre, matieres in matieres_par_semestre.items():
    data[semestre] = "\n".join(matieres)



# Sauvegarder dans un fichier CSV
with open('./masters.csv', 'a', newline='', encoding='utf-8') as csvfile:
    # Check if the file is empty
    file_is_empty = os.stat('./masters.csv').st_size == 0

    # Create a DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=data.keys())
    
    # Write the header only if the file is empty
    if file_is_empty:
        writer.writeheader()
    
    # Write the flattened result as a row
    writer.writerow(data)

 

C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\3662521263.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  objectifs_section = soup.find('h3', text="Objectifs")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\3662521263.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  competences_section = soup.find('h3', text="Compétences")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\3662521263.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  metiers_section = soup.find('h3', text="Métiers")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_9800\3662521263.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  secteurs_section = soup.find('h3', text="Secteurs d’activité")


In [28]:
import json
import os
import codecs

# Write to JSON
file_path = './masters.json'

# Check if the file exists
file_exists = os.path.isfile(file_path)

# Read existing data if the file exists
if file_exists:
    with codecs.open(file_path, 'r', encoding='utf-8-sig') as jsonfile:
        prev_data = json.load(jsonfile)
else:
    prev_data = []

# Append the new flattened result
prev_data.append(data)
print(prev_data)
# Write the updated data back to the JSON file
with open(file_path, 'w', encoding='utf-8') as jsonfile:
    json.dump(prev_data, jsonfile, ensure_ascii=False, indent=4)

[{'Objectifs': 'Le Master Professionnel en Business Analytics est co-construit avec l’entreprise « Business & Decision Tunisie ». Il est au croisement de la Data Science, de la Business Intelligence et du Management. Le programme vise à former des experts capables de développer des solutions décisionnelles analytiques.\nLe Master Professionnel en Business Analytics est disponible en double diplôme avec le Master Intelligence Artificielle & Business transformation avec PSTB.', 'Contenu': 'DATA SCIENCE ET INTELLIGENCE ARTIFICIELLE : Machine Learning, Deep Learning, Data Mining, Time Series, Statistical Analysis, Data Visualization.\nINFORMATIQUE : Big Data, Business Intelligence, Data Bases, Programmation\nMANAGEMENT : Business Process Management, E-business, Management Stratégique, Management de l’innovation.\nSOFT SKILLS\xa0 : Design Thinking, Séminaires, Développement Personnel, Langues.\nPROJET PROFESSIONNEL : Stages, PFE, Projets Intégrés, Visites d’entreprises, etc.', 'Compétences'